# Sistemas Lineares

Sistemas lineares são extremamentes importantes. Em quase todas aplicações avançadas teremos sistemas lineares.
Em muitos casos, esses sistemas serão especiais, de modo que teremos métodos especiais para resolvê-los.
Estudaremos aqui, no entanto, a maneira mais geral de resolvê-los. Estudos mais avançados serão feitos em disciplinas posteriores.

## A barra invertida

Em Julia e Matlab/Octave, existe um comando especial e simples para se resolver sistemas lineares: a barra invertida `\`.

In [1]:
A = rand(3, 3)
b = A * ones(3)
# b foi criado para que Ax = b tenha como solução o vetor de uns.
x = A\b

3-element Array{Float64,1}:
 1.0
 1.0
 1.0

A barra invertida facilita grandemente a resolução de sistemas. No entanto, é importante, principalmente para matemáticos industriais, aplicados e computacionais, entender o que acontece na resolução de um sistema. Além disso, também existem vários casos em que o uso da barra invertida será pior que o conhecimento específico sobre o problema.

## Eliminação Gaussiana

O método mais simples de resolução de sistemas lineares é o equivalente ao chamado escalonamento.

Relembrando. Dado uma matriz $A$ e um vetor $b$, o escalonamento da matriz aumentada
$\left[\begin{array}{cc} A & b \end{array}\right]$ é o processo de transformar essa matriz em uma matriz com vários zeros montando um tipo de escada. Com mais detalhes, é uma matriz tal que 
- Todas as linhas começam com um ou mais zeros que a linha de cima, a não que seja toda nula;
- Todas as linhas nulas estão abaixo de qualquer linha não toda nula.

Uma matriz é transformada numa matriz escalonada através de operações elementares nas linhas da matriz. Essa operações são
- multiplicar uma linha por um número não-nulo;
- trocar duas linhas de posição;
- somar à uma linha um múltiplo de outra.

Se $A$ é uma matriz quadrada e inversível, então é garantido que a matriz aumentada escalonada será da forma
$\left[\begin{array}{cc}U & c
\end{array}\right]$,
onde $C$ é uma matriz triangular superior com diagonal não nula, isto é
$c_{i,j} = 0$ se $i > j$, e $c_{i,i} \neq 0$. Em outras palavras, a primeira linha é não nula, e cada linha abaixo tem um zero a mais que a anterior.

In [2]:
# Exemplo
U = triu(rand(1:999, 10, 10)) / 1000

10×10 Array{Float64,2}:
 0.425  0.679  0.013  0.161  0.215  0.944  0.344  0.341  0.619  0.192
 0.0    0.835  0.621  0.174  0.391  0.985  0.549  0.804  0.861  0.855
 0.0    0.0    0.309  0.249  0.236  0.877  0.467  0.338  0.163  0.382
 0.0    0.0    0.0    0.502  0.321  0.004  0.946  0.401  0.285  0.137
 0.0    0.0    0.0    0.0    0.201  0.449  0.067  0.446  0.511  0.628
 0.0    0.0    0.0    0.0    0.0    0.1    0.588  0.954  0.351  0.872
 0.0    0.0    0.0    0.0    0.0    0.0    0.539  0.662  0.537  0.788
 0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.627  0.848  0.571
 0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.955  0.014
 0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.251

Computacionalmente, vamos fazer as mesmas operações, com exceção da multiplicação de uma linha por um número, por motivos que ficarão claros no futuro.
Além disso, o motivo para se mudar as linhas ao fazer essas operações na mão era para facilitar as contas.
Computacionalmente, teremos outros objetivos, e só faremos a mudança de linhas quando necessário.

Portanto, nossa operação principal será a adição de um múltiplo de outra linha a linha atual.
Nossa notação será

$$ L_j \leftarrow L_j + \alpha L_i, $$

para dizer que multiplicamos a linha $i$ por $\alpha$, somamos à linha $j$ e substituímos na linha $j$.

Vamos exemplificar o processo que já conhecemos.

In [3]:
A = [3.0 1 2; -1 2 1; 1 1 4]

3×3 Array{Float64,2}:
  3.0  1.0  2.0
 -1.0  2.0  1.0
  1.0  1.0  4.0

In [4]:
b = [6.0; 2; 6]

3-element Array{Float64,1}:
 6.0
 2.0
 6.0

In [5]:
m21 = A[2,1] / A[1,1]

-0.3333333333333333

In [6]:
# Li = B[i,:]
# L₂ ← L₂ - m₂₁L₁
A[2,:] = A[2,:] - m21 * A[1,:]
b[2] = b[2] - m21 * b[1]

4.0

Note que A[2,1] é zero agora

In [7]:
[A b]

3×4 Array{Float64,2}:
 3.0  1.0      2.0      6.0
 0.0  2.33333  1.66667  4.0
 1.0  1.0      4.0      6.0

In [8]:
# L₃ ← L₃ - m₃₁L₁
m31 = A[3,1]/A[1,1]
A[3,:] = A[3,:] - m31*A[1,:]
b[3] = b[3] - m31*b[1]

4.0

In [9]:
[A b]

3×4 Array{Float64,2}:
 3.0  1.0       2.0      6.0
 0.0  2.33333   1.66667  4.0
 0.0  0.666667  3.33333  4.0

In [10]:
# L₃ ← L₃ - m₃₂L₂
m32 = A[3,2]/A[2,2]
A[3,:] = A[3,:] - m32*A[2,:]
b[3] = b[3] - m32*b[2]

2.8571428571428568

In [11]:
[A b]

3×4 Array{Float64,2}:
 3.0   1.0          2.0      6.0    
 0.0   2.33333      1.66667  4.0    
 0.0  -1.11022e-16  2.85714  2.85714

Agora podemos resolver esse sistema triangular superior facilmente.
Note que, tirando o erro numérico em `A[3,2]`, temos

\begin{align}
a_{11} x_1 + a_{12} x_2 + a_{13} & = b_1 \\
a_{22} x_2 + a_{23} x_3 & = b_2 \\
a_{33} x_3 & = b_3.
\end{align}

Assim, resolvemos esse sistema fazendo

\begin{align}
x_3 & = \frac{b_3}{a_{33}} \\
x_2 & = \frac{b_2 - a_{23}x_3}{a_{22}} \\
x_1 & = \frac{b_1 - a_{13}x_3 - a_{12}x_2}{a_{11}}.
\end{align}

In [12]:
x = zeros(3)
x[3] = b[3] / A[3,3]
x[2] = (b[2] - A[2,3] * x[3]) / A[2,2]
x[1] = (b[1] - A[1,3] * x[3] - A[1,2] * x[2]) / A[1,1]
x

3-element Array{Float64,1}:
 1.0
 1.0
 1.0

O processo todo pode ser separado em duas partes:
- A redução à matriz triangular superior, que chamamos de **Eliminação Gaussiana**; e
- A resolução do sistema triangular superior.

O seguinte algoritmo descreve o processo

In [13]:
#=
Entrada: matriz A: n×n e vetor b: n
1. Para j de 1 a n-1
    1. Para i de j+1 a n
        1. Calcule mᵢⱼ = aᵢⱼ/aⱼⱼ
        2. Faça Lᵢ ← Lᵢ - mᵢⱼLⱼ
        3. Faça bᵢ ← bᵢ - mᵢⱼbⱼ
Saída: A triangular superior e 
       b com modificações correspondentes.
=#

Note que para ser bem definido, esse algoritmo precisa que $a_{jj} \neq 0$. Existe um teorema indicando as condições para isso. Antes, definimos $A_{kk}$ como a submatriz de $A$ das linhas 1 a $k$ e colunas 1 a $k$.
Em Julia seria a matriz `A[1:k,1:k]`.

**Teorema:** Se $\det(A_{kk}) \neq 0$ para todo $k = 1,\dots,n-1$, então o método de Eliminação Gaussiana como descrito acima está bem definida, isto é, roda todas as iterações até encontrar uma matriz triangular superior.
Se, além disso, $\det(A) \neq 0$, então a diagonal é não-nula. Caso contrário, o último elemento da diagonal será 0.

Como isso nem sempre será possível, vamos colocar essa condição para falha no algoritmo. Se o elemento da diagonal for muito próximo de zero, retornaremos um erro.

In [14]:
function elim_gauss(A::Matrix, b::Vector; diagtol = 1e-12)
    n = length(b)
    for j = 1:n-1
        ajj, bj = A[j,j], b[j]
        if abs(ajj) <= diagtol
            error("Diagonal muito próxima de 0")
        end
        # Implemente
    end
    return A, b
end

elim_gauss (generic function with 1 method)

In [15]:
A = [3.0 1 2; -1 2 1; 1 1 4]
b = [6.0;2;6]
elim_gauss(A, b)

([3.0 1.0 2.0; -1.0 2.0 1.0; 1.0 1.0 4.0], [6.0, 2.0, 6.0])

Agora para o algoritmo de resolução do sistema triangular superior.

In [16]:
#=
Entrada: A: n×n triangular superior com diagonal não-nula
         b: n
1. Crie o vetor x nulo
2. Para i de n à 1
    1. s ← bᵢ
    2. Para j de i+1 à n
        1. s ← s - aᵢⱼxⱼ
    3. xᵢ = s/aᵢᵢ

Saída: x: solução de Ax = b
=#

In [17]:
function sist_tri_sup(A, b)
    # Implemente
end

sist_tri_sup (generic function with 1 method)

In [18]:
A = [3.0 1 2; -1 2 1; 1 1 4]
b = [6.0;2;6]
elim_gauss(A, b)
sist_tri_sup(A, b)

## Pontos importantes

- $A$ foi perdida. Ela é irrecuperável (sem os valores de $m_{i,j})$. Para prevenir isso, podemos
    - Sol. 1: Não "usar" $A$, i.e., copiar para outra matriz
    - Sol. 2: Guardar $m_{i,j}$.
    - Sol. 3: Ambos
- A eliminação do elemento $a_{i,j}$ faz contas desnecessárias com 0.0 (antes da coluna $j$).
- $m_{i,j}$ foi calculado com o objetivo de transformar
$a_{i,j}$ em 0.0. Então não é necessário calcular a atualização de $a_{i,j}$. (Basta fazer `A[i,j] = 0.0`).
Sem falar que o resultado pode não ser exatamente 0.0, e
ficar sujando as contas.
- A matriz resultante tem muitos zeros, i.e., gasto de memória a toa.
    - Sol.: Guardar a matriz de um jeito diferente;
    - Sol.: Usar esse espaço vazio.
- `A[i,:]` cria um vetor temporario, então
$L_i \leftarrow L_i - m_{i,j}L_j$ tem um gasto de memória desnecessário.
    - Nesta disciplina, vamos ignorar esse fato, sempre que não afetar demais;
    - A maneira tradicional de resolver esse problema é usar `for`;
    - O Julia permite alguns comandos que fazem isso automático pra você.

Uma melhoria no método, que zera os elementos que deveriam ser zerados e restringe as contas feitas, está abaixo.

In [19]:
function elim_gauss(A::Matrix, b::Vector; diagtol = 1e-12)
    n = length(b)
    for j = 1:n-1
        ajj, bj = A[j,j], b[j]
        if abs(ajj) <= diagtol
            error("Diagonal muito próxima de 0")
        end
        for i = j+1:n
            mij = A[i,j] / ajj
            A[i,j] = 0.0
            A[i,j+1:n] -= mij * A[j,j+1:n]
            b[i] -= mij * bj
        end
    end
    return A, b
end

elim_gauss (generic function with 1 method)

In [20]:
A = [3.0 1 2; -1 2 1; 1 1 4]
b = [6.0;2;6]
elim_gauss(A, b)
A

3×3 Array{Float64,2}:
 3.0  1.0      2.0    
 0.0  2.33333  1.66667
 0.0  0.0      2.85714

## Pivoteamento

Pivoteamento é o nome dado à troca de linhas no processo de eliminação Gaussiana.
Às vezes podemos fazer troca de colunas também, e para diferenciar costuma-se dizer
eliminação Gaussiana com pivoteamento parcial.

O primeiro motivo pelo qual o pivoteamento é importante é o zero na diagonal.

In [21]:
A = [0 1; 1 0]

2×2 Array{Int64,2}:
 0  1
 1  0

Essa matriz, apesar de trivial, não passa no teste da nossa função de eliminação Gaussiana.
Além disso, existem outros problemas.

In [22]:
ϵ = 1e-12
A = [ϵ 100; 1.0 ϵ]
B = copy(A)
b = A * ones(2)
elim_gauss(A, b, diagtol=0.0)

([1.0e-12 100.0; 0.0 -1.0e14], [100.0, -1.0e14])

In [23]:
x = sist_tri_sup(A, b)
x - ones(2)

LoadError: [91mMethodError: no method matching -(::Void, ::Array{Float64,1})[0m
Closest candidates are:
  -([91m::SparseMatrixCSC[39m, ::Array) at sparse/sparsematrix.jl:1463
  -([91m::AbstractSparseArray{Tv,Ti,1} where Ti where Tv[39m, ::Union{Base.ReshapedArray{T,1,A,MI} where MI<:Tuple{Vararg{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int64},N} where N} where A<:DenseArray, DenseArray{T,1}, SubArray{T,1,A,I,L} where L} where I<:Tuple{Vararg{Union{Base.AbstractCartesianIndex, Int64, Range{Int64}},N} where N} where A<:Union{Base.ReshapedArray{T,N,A,MI} where MI<:Tuple{Vararg{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int64},N} where N} where A<:DenseArray where N where T, DenseArray} where T) at sparse/sparsevector.jl:1334
  -([91m::AbstractArray[39m, ::AbstractArray) at arraymath.jl:37
  ...[39m

In [24]:
A = B[[2;1], :]
b = A * ones(2)
elim_gauss(A, b)
x = sist_tri_sup(A, b)
x - ones(2)

LoadError: [91mMethodError: no method matching -(::Void, ::Array{Float64,1})[0m
Closest candidates are:
  -([91m::SparseMatrixCSC[39m, ::Array) at sparse/sparsematrix.jl:1463
  -([91m::AbstractSparseArray{Tv,Ti,1} where Ti where Tv[39m, ::Union{Base.ReshapedArray{T,1,A,MI} where MI<:Tuple{Vararg{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int64},N} where N} where A<:DenseArray, DenseArray{T,1}, SubArray{T,1,A,I,L} where L} where I<:Tuple{Vararg{Union{Base.AbstractCartesianIndex, Int64, Range{Int64}},N} where N} where A<:Union{Base.ReshapedArray{T,N,A,MI} where MI<:Tuple{Vararg{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int64},N} where N} where A<:DenseArray where N where T, DenseArray} where T) at sparse/sparsevector.jl:1334
  -([91m::AbstractArray[39m, ::AbstractArray) at arraymath.jl:37
  ...[39m

A matriz $A$ no exemplo acima
$$ A = \left[\begin{array}{cc}
\epsilon & 100 \\
1 & \epsilon
\end{array}\right],$$
introduz um problema durante a eliminação Gaussiana.
O elemento $a_{22}$ deve ser substituído por
$$ \epsilon - \frac{100}{\epsilon}. $$
Quando $\epsilon$ é pequeno, ocorre um underflow considerável nessa operação, de modo que fica aproximada
por $-\dfrac{100}{\epsilon}$.

O vetor $b$, que inicialmente é
$\left[\begin{array}{c} 100 + \epsilon \\ 1 + \epsilon\end{array}\right],$
vira
$\left[\begin{array}{c} 100 + \epsilon \\ 1 + \epsilon - \dfrac{100 + \epsilon}{\epsilon} \end{array}\right],$

Nem $100 + \epsilon$, nem $1 + \epsilon$ têm erros grandes. Por outro lado,
$$ 1 + \epsilon - \frac{100 + \epsilon}{\epsilon} = \epsilon - \frac{100}{\epsilon} \approx -\frac{100}{\epsilon}.$$

Ou seja, acaba acontecendo um erro similar em $b_2$, de modo que o $x_2$ encontrado é $1.0$.

Daí,
$$ x_1 = \frac{ (100 + \epsilon) - 100 }{\epsilon}. $$
A subtração tem um erro pequeno, mas quando dividido por $\epsilon$, fica considerável.

A matriz com as linhas trocadas, no entanto, não teve esse problema.

$$ A = \left[\begin{array}{cc}
1 & \epsilon \\
\epsilon & 100
\end{array}\right]
\qquad
\mbox{e}
\qquad
b = \left[\begin{array}{c}
1 + \epsilon \\
100 + \epsilon
\end{array}\right]
.$$

Temos $$ m_{21} = \epsilon, $$
daí
$$ a_{22} \leftarrow 100 - \epsilon^2 \approx 100, $$
e
$$ b_2 \leftarrow (100 + \epsilon) - \epsilon (1 + \epsilon) \approx 100. $$

Novamente $x_2 = 1$. Agora
$$ x_1 = \frac{ (1 + \epsilon) - \epsilon }{1} = 1. $$
Novamente o erro na subtração é grande, porém a divisão não aumenta esse erro.

Em outras palavras, podemos tentar pivotear a matriz $A$ para **aumentar a estabilidade numérica**.
Outro motivo para o pivoteamento está abaixo.

### Esparsidade

Uma matriz é dita **esparsa** se contém muitos zeros. Esse termo não é exato, o que complica um pouco, e não faz muito sentido para matrizes pequenas, mas vamos tentar elucidá-lo e explicar a sua importância.

A densidade de $A$ é o número de elementos não nulos divido pelo número de elementos total de $A$.

Em geral, o limite para se considerar uma matriz esparsa não é estabelecido, principalmente por depender da dimensão $n$ de $A$. Em geral, uma matriz com 1% de elementos não nulos é considerada esparsa. Uma com 10% às vezes será considerada esparsa. Uma matriz diagonal, por exemplo, é a matriz mais esparsa possível que pode ser não-singular (permutações da matriz diagonal também). O número de elementos não-nulos da matriz diagonal é $n$, e seu número de elementos é $n^2$. Portanto, sua densidade é $1/n$. Se $n > 1000$, então a densidade será menor que $0.001$.

Uma matriz tridiagonal - onde só podem ser não-nulos os elementos na diagonal, na diagonal acima, e na diagonal abaixo, terá densidade $\dfrac{3n-2}{n^2}$. No caso $n = 1000$, será $\approx 0.003$, e ainda é considerada esparsa.

Uma matriz de banda $k$ - onde as diagonais até $k$ acima e até $k$ abaixo da diagonal principal contém todos os elementos não-nulos - tem $n + 2(n-1) + 2(n-2) + \dots 2(n-k) = n + 2\sum_{i = 1}^k(n-i) = n(1 + 2k) - k(k+1)$
elementos. Uma matriz com $n = 1000$ e $k = 50$, terá esparsidade $\approx 0.1$, que ainda está bom. Veja que já é 10%.

Quando fazemos operações com matrizes esparsas, os cálculos envolvendo os elementos nulos tem custo nulo, além de não produzirem erros. Além disso, uma matriz esparsa irá ocupar muito menos espaço no computador.
Por isso matrizes esparsas são desejáveis.
Por outro lado, algumas operações na matriz fazem com que sua densidade aumente, por isso devemos tomar muito cuidade em como fazê-las.

Em particular, uma matriz esparsa pode se tornar bastante densa na operação de eliminação Gaussiana.

In [25]:
n = 10
A = eye(n)
A[:,1] = 1
A[1,:] = 1
A[1,1] = n
A

10×10 Array{Float64,2}:
 10.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
  1.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
  1.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
  1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0

In [26]:
b = A * ones(n)
elim_gauss(A, b)
A

10×10 Array{Float64,2}:
 10.0  1.0   1.0        1.0       …   1.0        1.0        1.0     
  0.0  0.9  -0.1       -0.1          -0.1       -0.1       -0.1     
  0.0  0.0   0.888889  -0.111111     -0.111111  -0.111111  -0.111111
  0.0  0.0   0.0        0.875        -0.125     -0.125     -0.125   
  0.0  0.0   0.0        0.0          -0.142857  -0.142857  -0.142857
  0.0  0.0   0.0        0.0       …  -0.166667  -0.166667  -0.166667
  0.0  0.0   0.0        0.0          -0.2       -0.2       -0.2     
  0.0  0.0   0.0        0.0           0.75      -0.25      -0.25    
  0.0  0.0   0.0        0.0           0.0        0.666667  -0.333333
  0.0  0.0   0.0        0.0           0.0        0.0        0.5     

A matriz $A$ acima é o mais densa possível (para uma matriz triangular superior).
No entanto, uma simples mudança de linhas resolveria o problema.

In [27]:
n = 10
A = eye(n)
A[:,1] = 1
A[1,:] = 1
A[1,1] = n
A[[1;n],:] = A[[n;1],:]
A

10×10 Array{Float64,2}:
  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
  1.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
  1.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
  1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 10.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0

In [28]:
elim_gauss(A, b)
A

10×10 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   1.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  -1.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  -1.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  -1.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  -1.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  -1.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  -1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  -1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  -1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  -1.0

Esse tipo de pivoteamento é melhora a esparsidade, e costuma de chamado de **pivoteamento para reduzir o preenchimento**.

### Como fazer o pivoteamento

Infelizmente, o pivoteamento para reduzir o preenchimento é complicado. Ele envolve procedimentos mais avançados e com menos garantias de que vai funcionar.
Por outro lado, o pivoteamento para melhorar a estabilidade numérica tem procedimentos mais básicos com verificações imediatas.
Desse modo, iremos focar nesse tipo de pivoteamento.

O pivoteamento para melhor a estabilidade numérica é feita escolhendo a linha a partir do **maior elemento em módulo da coluna atual** do método. Esse elemento, divo **pivô** é escolhido para ficar na diagonal da matriz, de modo que todos os $m_{ij}$ calculados satisfazem $|m_{ij}| \leq 1$. O processo está descrito abaixo.

In [29]:
#=
Entrada: matriz A: n×n e vetor b: n

1. Para j de 1 a n-1
    1. Encontra a linha k = argmin {|aᵢⱼ| : i = j,…,n} onde fica o pivô.
    2. Faça Lₖ ↔ Lⱼ
    3. Faça bₖ ← bⱼ
    4. Para i de j+1 a n
        1. Calcule mᵢⱼ = aᵢⱼ/aⱼⱼ
        2. Faça Lᵢ ← Lᵢ - mᵢⱼLⱼ
        3. Faça bᵢ ← bᵢ - mᵢⱼbⱼ

Saída: A triangular superior e 
       b com modificações correspondentes.
=#

A troca de linhas, como já feito anteriormente, é dada por

In [30]:
A = rand(5,5)
j, k = 2, 5
A[[k;j],:] = A[[j;k],:]

2×5 Array{Float64,2}:
 0.917908  0.262372  0.565862  0.477504  0.398566
 0.176515  0.270038  0.33932   0.91767   0.158029

In [31]:
function elim_gauss_pivot(A::Matrix, b::Vector; diagtol = 1e-12)
    n = length(b)
    for j = 1:n-1
        # Implemente o pivoteamento
        ajj, bj = A[j,j], b[j]
        if abs(ajj) <= diagtol
            error("Diagonal muito próxima de 0")
        end
        for i = j+1:n
            mij = A[i,j] / ajj
            A[i,j] = 0.0
            A[i,j+1:n] -= mij * A[j,j+1:n]
            b[i] -= mij * bj
        end
    end
    return A, b
end

elim_gauss_pivot (generic function with 1 method)

O processo de pivoteamento tem vantagens teóricas também. Note que o maior elemento do restante da coluna é escolhido. Isso quer dizer que se o pivô for nulo (ou suficientemente próximo), então realmente não existe a possibilidade de escalonar aquela matriz com a diagonal diferente de zero.
Em outras palavras, se $A$ é inversível, então a eliminação Gaussiana com pivoteamento deveria funcionar (na aritmética exata).

**Teorema:** Se $\det(A) \neq 0$, então a eliminação Gaussiana com pivoteamento gera uma matriz $A$ com diagonal não nula.

## Complexidade

Vamos analisar agora a complexidade da eliminação Gaussiana. Vamos recopiar a última versão aqui para referência.

In [32]:
#=
Entrada: matriz A: n×n e vetor b: n

1. Para j de 1 a n-1
    1. Encontra a linha k = argmin {|aᵢⱼ| : i = j,…,n} onde fica o pivô.
    2. Faça Lₖ ↔ Lⱼ
    3. Faça bₖ ↔ bⱼ
    4. Para i de j+1 a n
        1. Calcule mᵢⱼ = aᵢⱼ/aⱼⱼ
        2. Faça Lᵢ ← Lᵢ - mᵢⱼLⱼ
        3. Faça bᵢ ← bᵢ - mᵢⱼbⱼ

Saída: A triangular superior e 
       b com modificações correspondentes.
=#

Primeiro o que é mais direto.

- 1.4.1: 1 divisão
- 1.4.2: Cada $a_{i\ell} \leftarrow a_{i\ell} - m_{i j} a_{j\ell}$ são 2 operações.
    Fazemos isso para $\ell = j+1,\dots,n$, logo são $n - j$ vezes, num total de 2(n-j) operações
- 1.4.3: Mais 2 operações.

Um total de $2(n-j)+3$ operações para cada $i$.

Agora o loop 1.4. Ele é feito para $i = j+1,\dots,n$, logo um total de
$[2(n-j)+3](n-j) = 2(n-j)^2 + 3(n-j)$ operações para cada $j$.

 Os passos 1.1-1.3 não muito se feitos corretamente.

O total $2(n-j)^2 + 3(n-j)$ é feito para cada $j = 1,\dots,n-1$, que dá

\begin{align}
\sum_{j=1}^{n-1}[2(n-j)^2 + 3(n-j)]
& = \sum_{j=1}^{n-1}(2j^2 + 3j) \\
& = 2\frac{(n-1)n(2n-1)}{6} + 3\frac{(n-1)n}{2} \\
& = \frac{2}{3}n^3 + \frac{1}{2}n^2 - \frac{7}{6}n.
\end{align}

Agora, a complexidade da resolução do sistema triangular.

In [33]:
#=
Entrada: A: n×n triangular superior com diagonal não-nula
         b: n
1. Crie o vetor x nulo
2. Para i de n à 1, voltando
    1. s ← xᵢ
    2. Para j de i+1 à n
        1. s ← s - aᵢⱼxⱼ
    3. xᵢ = s/aᵢᵢ

Saída: x: solução de Ax = b
=#

Essa é um pouco mais fácil de se calcular.
A operação 1.2.2 faz $2(n-j)$ operações para cada $j$ num total de $2(n-j) + 1$ por $j$.
Logo, temos

\begin{align}
\sum_{j=1}^n [2(n-j) + 1]
& = \sum_{j=0}^{n-1}[2j + 1] \\
& = 2\sum_{j=0}^{n-1}j + n \\
& = (n-1)n + n = n^2.
\end{align}

Portanto, para resolver do zero um sistema linear $Ax = b$ pela eliminação Gaussiana,
fazemos
$$ \frac{2}{3}n^3 + \frac{3}{2}n^2 - \frac{7}{6}n. $$

O mais importante nesse valor todo é o $n^3$. Isso porque ele será o termo que mais "pesa" nessa conta para $n$ grande. Dizemos que a complexidade é da ordem de $n^3$.

Note que a complexidade de multiplicar uma matriz por um vetor é de ordem $n^2$, que é a mesma ordem de resolver um sistema triangular superior. Resolver o sistema do zero, no entanto, é de ordem $n^3$, que é a mesma ordem de multiplicar duas matrizes $n$ por $n$.

# Decomposição LU

No ponto atual, conseguimos resolver um sistema linear, sobre algumas
condições. Temos um problema, no entanto, e se quisermos resolver
mais de um sistema linear, e não tivermos todas os vetores do lado direito de imediato?

Isso acontece com frequência em muitas aplicações. Em particular, em otimização linear e não-linear com restrições lineares, essa situação é bastante comum. Também para o cálculo de autovalores e autovetores de uma matriz, existe um método que faz isso. De maneira geral, é como se a matriz $A$ fosse uma parte fixa de um modelo e o vetor $b$ fosse uma parte móvel.

Se precisamos resolver muitos sistemas com a mesma matriz $A$,
e começarmos do zero todos eles, teremos uma complexidade de ordem $n^3$ **para cada iteração desse método**.
No entanto, se tivermos os valores de $m_{ij}$, podemos fazer as operações necessárias
apenas no vetor do lado direito.

Note que temos $\dfrac{(n-1)n}{2}$ elementos $m_{ij}$ para guardar.
Não por acaso, eles equivalem a cada um dos zeros da matriz $A$
final.
Sendo assim, podemos **utilizar a própria matriz $A$ para guardar $m_{ij}$.**

Note também, que precisamos guardar as permutações que foram feitas, pois as linhas do vetor do lado direito também precisam ser trocadas.
Por enquanto, para facilitar o entendimento, vamos considerar a eliminação sem pivoteamento novamente.

In [34]:
function declu(A::Matrix; diagtol = 1e-12)
    n = size(A, 2)
    for j = 1:n-1
        ajj = A[j,j]
        if abs(ajj) <= diagtol
            error("Diagonal muito próxima de 0")
        end
        for i = j+1:n
            mij = A[i,j] / ajj
            A[i,j] = mij
            A[i,j+1:n] -= mij * A[j,j+1:n]
        end
    end
    return A
end

declu (generic function with 1 method)

Também precisamos criar uma função para aplicar os valores de $m_{ij}$ à um vetor $b$.
No entanto, veja que podemos ir direto ao objetivo e aproveitar e resolver o sistema logo.
Note que a aplicação dos $m_{ij}$ modifica o vetor $b$, e além disso criamos um novo vetor $x$. Isso quer dizer que estamos
- perdendo o $b$ original; e
- gastando mais memória com $x$.

Podemos resolver um desses problemas. As opções são
1. Copiar o $b$ para o $x$, aplicar $m_{ij}$ no $x$ e resolver o sistema modificando o $x$ para ser a solução no fim; ou
2. Aplicar $m_{ij}$ em $b$, e resolver o sistema modificando o $b$ para ser a solução no fim.

A opção 1 resolve o problema de perder $b$. Ao fim teremos $b$ e um novo $x$ que será a solução de $Ax = b$.

A opção 2 resolve o problema de gastar memória. Perderemos $b$, mas teremos a solução sem gasto de memória adicional.

A segunda opção é melhor pois se quisermos manter $b$, podemos copiar $b$ antes de chamar a função.

In [35]:
#=
Entrada: A: n×n onde a parte triangular superior é corresponde ao resultado
    da eliminação Gaussiana, e a parte triangular inferior sem a diagonal
    corresponde aos valores mᵢⱼ
         b: n   que será modificado para guardar a solução do sistema.

1. Para j de 1 a n-1
    1. Para i de j+1 a n
        1. Faça bᵢ ← bᵢ - aᵢⱼbⱼ
2. Para 1 de n à 1, voltando
    1. s ← bᵢ
    2. Para j de i+1 à n
        1. s ← s - aᵢⱼbⱼ
    3. bᵢ = s/aᵢᵢ
=#

In [36]:
function resolvelu(A, b)
    n = length(b)
    for j = 1:n-1
        bj = b[j]
        for i = j+1:n
            b[i] -= A[i,j] * bj
        end
    end
    # Copiar implementação já feita
    for i = n:-1:1
        s = b[i]
        for j = i+1:n
            s -= A[i,j] * b[j]
        end
        b[i] = s / A[i,i]
    end
    return b
end

resolvelu (generic function with 1 method)

In [37]:
A = [3.0 1 2; -1 2 1; 1 1 4]
b = A * ones(3)

declu(A)
resolvelu(A, b)

3-element Array{Float64,1}:
 1.0
 1.0
 1.0

Vamos analisar a primeira parte de `resolvelu` com mais detalhes, pensando num caso 3x3.

In [38]:
#= x é o valor de saída
Começo x₁ = b₁, x₂ = b₂, x₃ = b₃
j = 1
  i = 2
    x₂ ← x₂ - m₂₁x₁
  i = 3
    x₃ ← x₃ - m₃₁x₁
j = 2
  i = 3
    x₃ ← x₃ - m₃₂x₂
=#

No final desse processo temos as seguintes relações

\begin{align}\left\{\begin{array}{rcl}
x_1 & = & b_1 \\
x_2 & = & b_2 - m_{21} x_1 \\
x_3 & = & b_3 - m_{31} x_1 - m_{32} x_2.
\end{array}\right.
\end{align}

Isso quer dizer que

\begin{align}\left\{
\begin{array}{rcrcrcl}
      x_1 & &           & &     & = & b_1 \\
m_{21}x_1 &+&       x_2 & &     & = & b_2 \\
m_{31}x_1 &+& m_{32}x_2 &+& x_3 & = & b_3
\end{array}\right.
\end{align}

Que corresponde ao sistema linear $Lx = b$, onde $L$ é a matriz

$$\left[\begin{array}{ccc}
1 & 0 & 0 \\
m_{21} & 1 & 0 \\
m_{31} & m_{32} & 1
\end{array}\right].$$

Isso quer dizer que fazer eliminação Gaussiana em $\left[\begin{array}{cc}A & b\end{array}\right]$ é equivalente a fazer eliminação Gaussiana apenas em $A$, guardar $m_{ij}$ em $L$ e calcular $L^{-1}b$. Note que $\det(L) = 1$ e por isso sempre existe a inversa.

Chamando a matriz resultante da eliminação Gaussiana em $A$ de $U$, podemos dizer que começamos com o sistema
$$ Ax = b, $$
e acabamos com o sistema
$$ Ux = L^{-1}b. $$

Mas então podemos fazer
$$ LUx = b. $$

Será que $A = LU$? A resposta é sim, pois os valores de $x$ e $b$ não são usados para se calcular nem $L$ nem $U$. Portanto, podemos mudar $x$ e obter um $b$ correspondente, e os dois sistemas continuarão válidos. Outra maneira de ver isso é subtrair os dois sistemas, obtendo $(A - LU)x = 0$, e variar $x$ para cada coluna da identidade.

In [39]:
A = rand(5, 5)
B = copy(A)
declu(A)

5×5 Array{Float64,2}:
 0.460624   0.982056   0.131722   0.761447    0.143639
 1.15977   -0.164407   0.310202  -0.618326    0.524453
 0.814343  -0.293935   0.265688   0.0136406   0.633328
 0.678402   4.01917   -2.56535    2.02607    -0.092565
 1.91953    9.72372   -8.5499     2.78058     1.00731 

In [40]:
L = tril(A, -1) + eye(5)

5×5 Array{Float64,2}:
 1.0        0.0        0.0      0.0      0.0
 1.15977    1.0        0.0      0.0      0.0
 0.814343  -0.293935   1.0      0.0      0.0
 0.678402   4.01917   -2.56535  1.0      0.0
 1.91953    9.72372   -8.5499   2.78058  1.0

In [41]:
U = triu(A)

5×5 Array{Float64,2}:
 0.460624   0.982056  0.131722   0.761447    0.143639
 0.0       -0.164407  0.310202  -0.618326    0.524453
 0.0        0.0       0.265688   0.0136406   0.633328
 0.0        0.0       0.0        2.02607    -0.092565
 0.0        0.0       0.0        0.0         1.00731 

In [42]:
L * U

5×5 Array{Float64,2}:
 0.460624  0.982056    0.131722  0.761447   0.143639
 0.534216  0.974548    0.462968  0.264774   0.691041
 0.375106  0.848055    0.281775  0.815466   0.596144
 0.312488  0.00544875  0.654532  0.0224852  0.48804 
 0.884181  0.286435    0.997556  0.966216   0.71039 

In [43]:
B - L * U

5×5 Array{Float64,2}:
 0.0   0.0           0.0          0.0          0.0        
 0.0   0.0           0.0          0.0          0.0        
 0.0   0.0           0.0          0.0          0.0        
 0.0   1.82146e-17   2.22045e-16  2.22045e-16  2.22045e-16
 0.0  -5.55112e-17  -1.11022e-16  2.22045e-16  8.88178e-16

Concluindo, o processo de eliminação Gaussiana nos dá uma maneira de representar $A$ que depende de duas matrizes mais simples. Chamamos essa maneira de **fatoração LU**.

Existem várias fatorações matriciais. Algumas são bem conhecidas, e algumas são bem específicas. A fatoração LU é a mais famosa, pois não depende de nenhuma propriedade especial da matriz para existe, com exceção do Teorema que passamos.

Dado a fatoração $LU$ da matriz $A$, não iremos mais pensar em resolver $Ax = b$ como a aplicação da eliminação Gaussiana, e sim como a resolução de sistemas triangulares.

Dado $Ax = b$, e $A = LU$, temos
$$ LUx = b. $$
Para resolver esse sistema, fazemos $Ux = y$, obtendo
$$ \left\{\begin{array}{rcl}
Ly & = & b \\
Ux & = & y.
\end{array}\right. $$

Dessa maneira, fica óbvio que resolvemos dois sistemas triangulares.

Para resolver o sistema triangular $Ly = b$, veja que
$$\left\{\begin{array}{rcl}
y_1 & = & b_1 \\
y_2 & = & b_2 - m_{21}y_1 \\
y_3 & = & b_3 - m_{31}y_1 - m_{32}y_2 \\
\vdots & & \vdots \\
y_n & = & b_n - \sum_{j=1}^{n-1} m_{nj} y_j.
\end{array}\right.$$

Em outras palavras,
$$ y_i = b_i - \sum_{j = 1}^{i - 1}m_{ij} y_j, \qquad i = 1,\dots,n. $$

Veja que se calculamos na ordem natural $1,\dots,n$, teremos os valores necessários para definir todos os $y_i$.

Abaixo vemos o algoritmo para resolver esse sistema, e vamos substituir $b$ de entrada pelo valor de $y$ de saída.

In [44]:
#=
Entrada: L: nxn triangular inferior com diagonal de 1
         b: n
1. Para i de 1 à n-1
    1. Para j de 1 à i-1
        1. bᵢ ← bᵢ - mᵢⱼ bⱼ
=#

Comparando com o algoritmo que faz a eliminação Gaussiana em $b$, vemos uma leva diferença

In [45]:
#= Algoritmo que faz elim. Gauss. em b
1. Para j de 1 a n-1
    1. Para i de j+1 a n
        1. Faça bᵢ ← bᵢ - aᵢⱼbⱼ
=#

A ordem desse algoritmo está diferente, e os limites para os loops de $i$ e $j$ também. No entanto, a operação que é calculada é a mesma (lembre-se que $a_{ij} = m_{ij}$).

### Fatoração LU com pivoteamento

Como vimos anteriormente, a qualidade da solução do sistema pela eliminação Gaussiana, muda drasticamente quanto utilizamos pivotamento. Na fatoração LU, isso também é verdade.

Agora que guardamos a matriz $L$, o pivoteamento também irá afetá-la. Além disso, queremos calcular a fatoração sem a necessidade de um vetor do lado direito, então precisamos guardar o pivoteamento feito para fazê-lo no vetor $b$ também.

Inicialmente, vamos apenas apresentar o algoritmo com pivoteamento.
Nesse algoritmo precisaremos criar um **vetor** para guardar a permutação das linhas.
Esse vetor será um vetor com a ordem que as linhas ficou no fim. Ele começa sendo um vetor $p_i = i$.

In [46]:
function declupivot(A::Matrix; diagtol = 1e-12)
    n = size(A, 2)
    p = collect(1:n)
    for j = 1:n-1
        # Quem é o pivô
        pivo, k = abs(A[j,j]), j
        for i = j+1:n
            if abs(A[i,j]) > pivo
                pivo, k = abs(A[i,j]), i
            end
        end
        if pivo <= diagtol
            error("Matriz singular ou muito próxima de ser singular")
        end
        
        if k != j
            p[k], p[j] = p[j], p[k]
            A[[k;j],:] = A[[j;k],:]
        end
        ajj = A[j,j]
        for i = j+1:n
            mij = A[i,j] / ajj
            A[i,j] = mij
            A[i,j+1:n] -= mij * A[j,j+1:n]
        end
    end
    return p
end

declupivot (generic function with 1 method)

O vetor $p$ corresponde à uma **matriz de permutação** $P$. Uma matriz de permutação é uma matriz obtida a partir de permutações de linhas e permutações de colunas na matriz identidade.
- Se a matriz $P$ é obtida a partir de uma sequência de permutações nas **linhas** da matriz identidade, então a matriz $PA$ é a matriz resultante das mesmas permutações nas linhas de $A$. Nesse caso, dizemos que $P$ é uma **matriz de permutação de linhas**.
- Análogo para colunas, e para a matriz $AP$.

O vetor $p$ tem os índices dessas linhas. Inicialmente, $p$ é um vetor com $p_i = i$, que significa que $P$ é a identidade. Cada modificação no passo XXX modifica elementos de $p$ e as linhas correspondentes de $P$.

Uma matriz de permutação de linhas também é uma matriz de permutação de colunas, mas com outras permutações (a priori indetermidanas).

**Exercício:** Mostre que se $P$ é uma matriz de permutação de linhas, então $P^T$ é uma matriz de permutação de colunas.

Uma parte não trivial do processo da fatoração LU com pivoteamento é descobrir a relação entre $A$, $L$, $U$ e $P$. Como não dispomos do tempo, vamos simplesmente enunciar essa relação.

**Teo.:** O algoritmo de fatoração LU com pivoteamento parcial da matriz $A$ retorna matrizes $L$, $U$ e $P$ tais que
$$ PA = LU. $$

In [47]:
A = rand(5, 5)
B = copy(A)
p = declupivot(A)
L = tril(A, -1) + eye(5)
U = triu(A)
L * U - B[p,:]

5×5 Array{Float64,2}:
  0.0          0.0  0.0          0.0  0.0
  0.0          0.0  0.0          0.0  0.0
  0.0          0.0  0.0          0.0  0.0
  0.0          0.0  1.38778e-17  0.0  0.0
 -5.55112e-17  0.0  0.0          0.0  0.0

Note ainda, que por um teorema anterior, a fatoração LU de uma matriz quadrada sempre existe. Se $A$ for singular, então $U$ terá um zero no último elemento.

## Exercícios

Exercícios do capítulo 3 do livro Cálculo Numérico de Ruggiero e Lopes, com exceção daqueles que não competem ao assunto.